In [3]:
import numpy as np
import h5py
import scipy

In [4]:
# Loading the data
with h5py.File('../HAR/preprocessed.hdf5','r') as hf:
    x_train = np.array(hf.get('x_train'))
    y_train = np.array(hf.get('y_train'))
    s_train = np.array(hf.get('s_train'))
    x_test = np.array(hf.get('x_test'))
    y_test = np.array(hf.get('y_test'))
    s_test = np.array(hf.get('s_test'))
    x_train_with_past = np.array(hf.get('x_train_with_past'))
    y_train_with_past = np.array(hf.get('y_train_with_past'))
    x_test_with_past = np.array(hf.get('x_test_with_past'))
    y_test_with_past = np.array(hf.get('y_test_with_past'))

In [8]:
s_train

array([ 1,  1,  1, ..., 30, 30, 30])

In [9]:
y_train

array([5, 5, 5, ..., 2, 2, 2])

In [10]:
np.sum(s_train == 1)

347

In [285]:
np.sum(transition_train, axis=0)

array([ 1.00833789,  0.99222336,  0.99488295,  0.99715828,  1.00285639,
        1.00454112])

In [310]:
# Learning a one component Gaussian over all the features
def compute_transition(y, alpha=0.1):
    '''
    Compute the transition matrice.
    Rows: states to
    cols: states from
    States are indexed starting from 1
    '''
    num_state = np.max(y)
    transition = alpha*np.ones((num_state, num_state))
    for i in xrange(y.shape[0]-1):
        transition[y[i+1]-1, y[i]-1] += 1
    # Normalisation (column should be normalized)
    transition /= np.sum(transition, axis=0)[:, np.newaxis]
    return transition

def compute_emission(x, y):
    '''
    Compute the parameters of the gaussian distribution
    of the emission given each state.
    We assume each emission distribution is independent,
    the covariance matrix is diagonal then.
    States are indexed starting from 1
    '''
    num_state = np.max(y)
    
    sigma_diag = np.zeros((num_state, x.shape[1]))
    mu = np.zeros((num_state, x.shape[1]))
    for s in xrange(num_state):
        x_s = x[(y == s+1), :]
        # Computing mu_s
        mu[s] = np.mean(x_s, axis=0)
        # Computing sigma_s (by column)
        sigma_diag[s] = np.std(x_s, axis=0)

    return mu, sigma_diag

def compute_logscore(data, log_transition, mu, sigma, C):
    y = np.zeros((C, C))
    for j in xrange(C):
        y[j, :] = np.log(compute_logB(data, mu, sigma_diag, j))

    return y + log_transition

def compute_logscore_pymc(data, log_transition, means, cov, C):
    y = np.zeros((C, C))
    for j in xrange(C):
        y[j, :] = scipy.stats.multivariate_normal.logpdf(data, mean=means[j], cov=covs[j])
    return y + log_transition

def viterbi(inputs, init, log_transition, mu, sigma, C):
    '''
    Evaluates the highest scoring sequence
    '''
    y = np.zeros((C, C))
    initial = np.zeros(C)

    initial[init] = 1
    initial = np.log(initial)

    n = inputs.shape[0]
    # To store the maxes
    max_table = np.zeros((n, C))
    backpointer_table = np.zeros((n, C))

    # first timestep
    # the initial most likely paths are the initial state distribution
    state_init = initial + compute_logscore(inputs[0,:], log_transition, mu, sigma, C)
    maxes = np.max(state_init, axis=1)
    backpointers = np.argmax(state_init, axis=1)
    max_table[0, :] = maxes

    for i in xrange(1, n):
        # COmputing the score
        y = compute_logscore(inputs[i, :], log_transition, mu, sigma, C)
        scores = y + np.repeat(maxes.reshape(1, C), C, axis=0)

        # compute new maxes
        maxes = np.max(scores, axis=1)
        backpointers = np.argmax(scores, axis=1)

        max_table[i, :] = maxes
        backpointer_table[i, :] = backpointers

    # follow backpointers to recover max path
    classes = np.zeros(n)
    classes[n-1] = np.argmax(maxes, axis=0)
    for i in xrange(n-1, 0, -1):
        classes[i-1] = backpointer_table[i, classes[i]]

    return classes

def standardize(x):
    '''
    Standardize each column of x
    '''
    x_std = np.std(x, axis=0)
    x_mu = np.mean(x, axis=0)
    
    return (x - x_mu)/x_std[np.newaxis, :]

def compute_accuracy(pred_classes, true_classes):
    '''
    Compute accuracy
    '''
    return np.sum(pred_classes == true_classes) /(1.*len(pred_classes))

def compute_logB(data_point, mu, sigma_diag, j):
    '''
    Compute log(p(x|s_j))
    '''
    return np.sum([scipy.stats.norm.logpdf(d, loc=mu[j, i], scale=sigma_diag[j, i]) for i, d in enumerate(data_point)])

def compute_B(data_point, mu, sigma_diag, j):
    '''
    Compute p(x|s_j)
    '''
    return np.prod([scipy.stats.norm.pdf(d, loc=mu[j, i], scale=sigma_diag[j, i]) for i, d in enumerate(data_point)])

In [176]:
C=6
init = 4
y = np.zeros((C, C))
initial = np.zeros(C)

initial[init] = 1
initial = np.log(initial)
print(initial)

state_init = initial + compute_logscore(x_standard[0,:], log_transition_train, mu, sigma_diag, C)
print(state_init)
print(np.max(state_init, axis=1))

[-inf -inf -inf -inf   0. -inf]
[[        -inf         -inf         -inf         -inf -10.65349168
          -inf]
 [        -inf         -inf         -inf         -inf -14.8395688
          -inf]
 [        -inf         -inf         -inf         -inf -12.82739194
          -inf]
 [        -inf         -inf         -inf         -inf  -7.88384383
          -inf]
 [        -inf         -inf         -inf         -inf  -0.84924053
          -inf]
 [        -inf         -inf         -inf         -inf -20.8956498
          -inf]]
[-10.65349168 -14.8395688  -12.82739194  -7.88384383  -0.84924053
 -20.8956498 ]


/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:7: RuntimeWarning: divide by zero encountered in log


## 1) Sequence Prediction

### Subsample + MLE

In [300]:
# We retain 6 features (known to be independent)

x = np.concatenate((x_train[:, :3], x_train[:, 41:44]), axis=1)
x_sub_test = np.concatenate((x_test[:, :3], x_test[:, 41:44]), axis=1)
print(x.shape)

(7352, 6)


In [429]:
# Learning the HMM

# standardization
x_standard = standardize(x)
print(x_standard.shape)

# ### TRANSITION
transition_train = compute_transition(y_train)
log_transition_train = np.log(transition_train)
print(transition_train.shape)

# ### EMISSION
mu, sigma_diag = compute_emission(x_standard, y_train)
print(mu.shape)
print(sigma_diag.shape)

(7352, 6)
(6, 6)
(6, 6)
(6, 6)


In [304]:
%%time
# Sequence prediction
C = 6
sample_size = 3000
seq_pred = viterbi(x_standard[:sample_size,:], 4, log_transition_train, mu, sigma_diag, C)
# Shifting the index of 1
seq_pred += 1
print 'ACCURACY train: {}'.format(compute_accuracy(seq_pred, y_train[:sample_size]))

/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:59: RuntimeWarning: divide by zero encountered in log
/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:41: RuntimeWarning: divide by zero encountered in log


ACCURACY train: 0.876666666667
CPU times: user 7.27 s, sys: 67.8 ms, total: 7.34 s
Wall time: 7.91 s


/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:89: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [293]:
%%time
x_sub_test_standard = standardize(x_sub_test)
seq_pred_test = viterbi(x_sub_test_standard[:sample_size,:], 4, log_transition_train, mu, sigma_diag, C)
seq_pred_test += 1
print 'ACCURACY test: {}'.format(compute_accuracy(seq_pred_test, y_test[:sample_size]))

/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:59: RuntimeWarning: divide by zero encountered in log
/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:89: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


ACCURACY test: 0.768917543264
CPU times: user 7.64 s, sys: 97.1 ms, total: 7.73 s
Wall time: 8.47 s


In [193]:
print seq_pred_test[:100]

[ 5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.
  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.
  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.
  5.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.
  6.  6.  6.  6.  6.  6.  6.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


### Subsample + pyMC

In [220]:
# We use here the model fited by PyMc
means = []
covs = []

with h5py.File('../HAR/means_covs.hdf5', "r") as f:
    for k in xrange(6):
        means.append(np.array(f.get('mu_{}'.format(k+1))))
        # Pymc provides tau, cov = (tau)^{-1}
        covs.append(np.linalg.pinv(np.array(f.get('cov_{}'.format(k+1)))))
means = np.array(means)
covs = np.array(covs)

In [221]:
%%time
# Sequence prediction
C = 6
sample_size = 1000
seq_pred = viterbi(x_standard[:sample_size,:], 4, log_transition_train, means, covs, C)
# Shifting the index of 1
seq_pred += 1
print 'ACCURACY train: {}'.format(compute_accuracy(seq_pred, y_train[:sample_size]))

/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:59: RuntimeWarning: divide by zero encountered in log
/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:89: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 16 µs
ACCURACY train: 0.278


### More features

# Forward Backward algorithm

In [322]:
print(compute_B(x[0], mu, sigma_diag, 4))
print(compute_logB(x[0], mu, sigma_diag, 4))

0.0364018687618
-3.31313516605


In [471]:
def forward(x, init, end, log_transition, mu, sigma_diag):
    '''
    Compute log(p(X|lambda)) with lambda the HMM parameters
    (here transition, mu, sigma_diag)
    alpha[t, j] = p(x_1, ... x_t, S(t) = s_j|lambda)
    NB:
        alpha[0, :] used only as initialization
        log-sum-exp trick used
    '''
    C = mu.shape[0]
    T = x.shape[0]
    # Initialization
    alpha = np.zeros((T+1, C))
    alpha[0, init] = 1
    alpha[0,:] = np.log(alpha[0,:])
    
    # Recursion
    for t in xrange(1, T-1):
        for j in xrange(C):
            b_t_j = compute_logB(x[t-1], mu, sigma_diag, j)
            a = b_t_j + alpha[t-1, :] + log_transition[j,:]
            # log-sum-exp trick
            max_a = np.max(a)
            alpha[t, j] = max_a + np.log(np.sum(np.exp(a - max_a)))
    
    # Termination
    b_t_j = compute_logB(x[T-1], mu, sigma_diag, end)
    a = b_t_j + alpha[T, :] + log_transition[:,end]
    max_a = np.max(a)
    alpha[T, end] = max_a + np.log(np.sum(np.exp(a - max_a)))
    
    return alpha

def backward(x, init, end, log_transition, mu, sigma_diag):
    '''
    Compute the log backward probabilities.
    beta[t, j] = p(x_{t+1}, ..., x_{T}| S(t) = s_j, lambda)
    '''
    C = mu.shape[0]
    T = x.shape[0]
    # Initialization
    beta = np.zeros((T+1, C))
    for i in xrange(C):
        beta[T, i] = log_transition[end, i]
    
    # Recursion
    for t in xrange(T-1, 0, -1):
        for j in xrange(C):
            a = beta[t+1, :] + log_transition[j,:]
            for i in xrange(C):
                b_t1_i = compute_logB(x[t], mu, sigma_diag, i)
                a[i] += b_t1_i
            max_a = np.max(a)
            beta[t, j] = max_a + np.log(np.sum(np.exp(a - max_a))) 
    
    # Termination
    a = beta[1, :] + log_transition[:, init]
    for i in xrange(C):
        b_1_i = compute_logB(x[0], mu, sigma_diag, i)
        a[i] += b_t1_i
    max_a = np.max(a)
    beta[0, init] = max_a + np.log(np.sum(np.exp(a - max_a))) 
        
    return beta

def compute_state_probability(alpha, beta, end):
    '''
    compute state occupation log probability
    '''
    gamma = alpha + beta
    # Normalization
    gamma -= alpha[-1, end]
    return gamma

def compute_state_transition(x, alpha, beta, log_transition, mu, sigma_diag, end):
    '''
    Compute log(p(S(t) = s_i, S(t+1) = s_j| X, lambda))
    '''
    T = x.shape[0]
    C = mu.shape[0]
    psi = np.zeros((T, C, C))
    for t in xrange(T):
        for j in xrange(C):
            b_t_j = compute_logB(x[t], mu, sigma_diag, j)
            for i in xrange(C):
                psi[t, i, j] = alpha[t, i] + log_transition[j, i] + beta[t+1, j] + b_t_j
    
    return psi - alpha[-1, end]
    
    
def forward_backward(x, init, end, log_transition, mu, sigma_diag, n_iterations):
    '''
    EM algorithm to fit an HMM to a sequence of observation.
    Take as argnument an initial HMM and returns a finer one.
    '''
    # E-step: estimate the state occupation probabilities (in LOG)
    log_alpha = forward(x, init, end, log_transition, mu, sigma_diag)
    log_beta = backward(x, init, end, log_transition, mu, sigma_diag)
    log_state_probability = compute_state_probability(alpha, beta, end)
    log_state_transition = compute_state_transition(x, alpha, beta, log_transition, mu, sigma_diag, end)
    
    # M-step: re-estimate HMM parameers
    state_probability = np.exp(log_state_probability)
    
    denom = np.sum(state_probability, axis=0)[np.newaxis, :]
    mu = np.dot(state_probability.T, x)/denom
    
    for j in xrange(C):
        diff = x - mu[j]
        sigma_diag[:, j] = np.std(np.multiply(np.sqrt(np.abs(state_probability[:,j])), diff), axis=1)
    sigma_diag /= denom
    
    state_transition = np.exp(log_state_transition)
    for i in xrange(C):
        for j in xrange(C):
            log_transition[i, j] = np.log(np.sum(state_transition[:, i, j]))
        log_transition[i,:] -= np.sum(state_transition[:, i, :])
    
    return log_transition, mu, sigma_diag
    
    

In [389]:
log_transition_train

array([[-0.03526312, -9.41458649, -9.41458649, -9.41458649, -9.41458649,
        -3.37195365],
       [-9.28042598, -0.05116572, -2.9870067 , -9.28042598, -9.28042598,
        -9.28042598],
       [-3.15421695, -4.24808989, -0.05897258, -9.19684978, -9.19684978,
        -9.19684978],
       [-9.46234345, -9.46234345, -9.46234345, -0.03439482, -3.41971062,
        -7.06444818],
       [-9.52850315, -3.55979559, -6.48398071, -9.52850315, -0.03140614,
        -9.52850315],
       [-9.5522265 , -9.5522265 , -9.5522265 , -3.48611841, -9.5522265 ,
        -0.03139126]])

In [472]:
%%time
alpha = forward(x_standard, 4, 1, log_transition_train, mu, sigma_diag)

CPU times: user 20 s, sys: 300 ms, total: 20.3 s
Wall time: 23 s


/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:15: RuntimeWarning: divide by zero encountered in log


In [475]:
alpha[-1, 1]

-13.383657281332981

In [432]:
%%time
beta = backward(x_standard, 4, 1, transition_train, mu, sigma_diag)

CPU times: user 1min 41s, sys: 528 ms, total: 1min 41s
Wall time: 1min 43s


In [433]:
beta

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,  -2.82130318e+04,   0.00000000e+00],
       [ -2.82039979e+04,  -2.82041351e+04,  -2.82041184e+04,
         -2.82040964e+04,  -2.82032426e+04,  -2.82041383e+04],
       [ -2.82045037e+04,  -2.82046236e+04,  -2.82046084e+04,
         -2.82045847e+04,  -2.82037219e+04,  -2.82046266e+04],
       ..., 
       [ -3.46044252e+00,  -3.92440437e+00,  -3.86841289e+00,
         -3.99421940e+00,  -3.50917685e+00,  -4.01322867e+00],
       [ -3.55068100e+00,  -2.93135336e+00,  -3.09425013e+00,
         -3.58311740e+00,  -3.56977062e+00,  -3.58753772e+00],
       [  9.32314003e-05,   9.50121201e-01,   5.04381876e-02,
          9.32314003e-05,   9.32314003e-05,   9.32314003e-05]])

In [434]:
%%time
log_state_probability = compute_state_probability(alpha, beta, 1)

CPU times: user 342 µs, sys: 560 µs, total: 902 µs
Wall time: 523 µs


In [435]:
log_state_probability

array([[            -inf,             -inf,             -inf,
                    -inf,   1.07081128e+04,             -inf],
       [  1.07064931e+04,   1.07021708e+04,   1.07041988e+04,
          1.07091643e+04,   1.07170520e+04,   1.06961106e+04],
       [  1.07048689e+04,   1.07005865e+04,   1.07024716e+04,
          1.07079809e+04,   1.07160041e+04,   1.06950842e+04],
       ..., 
       [  4.08235136e-01,   5.08212972e+00,   5.24975955e+00,
         -1.48447212e+00,   7.08567891e+00,  -1.22325067e+01],
       [ -8.73097611e-01,   2.11465479e+00,   2.61131359e+00,
         -2.42437828e+00,   5.63969140e+00,  -1.47016760e+01],
       [ -4.65002938e+00,   9.50121201e-01,   1.15931027e+00,
         -3.66143284e+00,  -1.74864808e+00,  -1.35500344e+01]])

In [447]:
t = 3
i=1
j=1
b_t_j = compute_logB(x[t], mu, sigma_diag, j)
print b_t_j
print(alpha[1, i] + log_transition_train[j, i] + beta[t+1, j] + b_t_j - alpha[-1, 1])

-10.0136295297
10690.6640337


In [459]:
%%time
log_state_transition = compute_state_transition(x_standard, alpha, beta, log_transition_train, mu, sigma_diag, 1)

CPU times: user 17.1 s, sys: 114 ms, total: 17.2 s
Wall time: 17.5 s


In [461]:
log_state_transition.shape

(7352, 6, 6)

In [ ]:
log_alpha = alpha
log_beta = beta

# M-step: re-estimate HMM parameers
state_probability = np.exp(log_state_probability)

denom = np.sum(state_probability, axis=0)[np.newaxis, :]
mu = np.dot(state_probability.T, x_standard)/denom

In [425]:
for j in xrange(C):
    diff = x_standard - mu[j]
    sigma_diag[:, j] = np.std(np.sqrt(np.abs(state_probability[:,j]))[:, np.newaxis]*diff, axis=0)
sigma_diag /= denom

state_transition = np.exp(log_state_transition)
for i in xrange(C):
    for j in xrange(C):
        log_transition[i, j] = np.log(np.sum(state_transition[:, i, j]))
    log_transition[i,:] -= np.sum(state_transition[:, i, :])

/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:5: RuntimeWarning: overflow encountered in exp
/Users/nicolasdrizard/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:15: RuntimeWarning: overflow encountered in exp


NameError: name 'log_transition' is not defined

In [424]:
np.std(np.sqrt(np.abs(state_probability[:,0]))[:, np.newaxis]*diff, axis=0).shape

(6,)

In [474]:
beta[0, 4]

-28213.031783239847

In [473]:
alpha[-1, 1]

-13.383657281332981